In [ ]:
# import julia

# julia.install()

In [ ]:
from julia import Main

In [ ]:
Main.eval(
    """
using DataFrames
using Pandas
using Plotly

using GCTGMT 
using FeatureSetEnrichment
"""
)

In [ ]:
import pandas as pd

In [ ]:
score_ = pd.read_csv(
    "/Users/kwat/code/jl/FeatureSetEnrichment.jl/notebook/data/gene_score.tsv",
    sep="\t",
    index_col=0,
    squeeze=True,
)

element_x_sample = pd.concat(
    (score_, score_ * 10, score_ / 10), axis=1
).reset_index()

element_x_sample.columns = (
    "Sample {}".format(i) for i in range(element_x_sample.shape[1])
)

set_to_element_ = Main.eval(
    """
read_gmt("/Users/kwat/code/jl/FeatureSetEnrichment.jl/notebook/data/c2.all.v7.1.symbols.gmt")
"""
)

set_element_ = set_to_element_["COLLER_MYC_TARGETS_UP"]

In [ ]:
def send(score_, set_element, set_to_element_, element_x_sample):

    if score_ is not None:

        Main.element_ = list(score_.index)

        Main.score_ = list(score_)

    if set_element_ is not None:

        Main.set_element_ = list(set_element_)

    if set_to_element_ is not None:

        Main.set_to_element_ = set_to_element_

        Main.eval(
            """
        set_to_element_ = convert(Dict{String, Vector{String}}, set_to_element_)
        """
        )

    if element_x_sample is not None:

        Main.element_x_sample = element_x_sample

        Main.eval(
            """
        element_x_sample = DataFrames.DataFrame(Pandas.DataFrame(element_x_sample))
        """
        )

In [ ]:
def score_1_sample_1_set(score_, set_element_, sort=True, plot=True, title_text = "Title"):

    send(score_, set_element_, None, None)

    Main.sort = sort

    Main.plot = plot
    
    Main.title_text = title_text
    
    result = Main.eval(
        """
    score_set(
        element_,
        score_,
        set_element_;
        sort = sort,
        plot = plot,
        title_text = title_text,
    )
    """
    )

    return result


score_1_sample_1_set(score_, set_element_)

In [ ]:
def score_1_sample_n_set(score_, set_to_element_, sort=True):

    send(score_, None, set_to_element_, None)

    Main.sort = sort

    set_to_retsult = Main.eval(
        """
    score_set(
        element_,
        score_,
        set_to_element_;
        sort = sort,
    )
    """
    )

    return set_to_retsult

score_1_sample_n_set(score_, set_to_element_)

In [ ]:
def score_n_sample_n_set(element_x_sample, set_to_element_, n_job=1):

    send(None, None, set_to_element_, element_x_sample)

    Main.n_job = n_job

    set_x_sample = Main.eval(
        """
    Pandas.DataFrame(score_set(
        element_x_sample,
        set_to_element_;
        n_job = n_job,
    ))
    """
    )

    return set_x_sample.set_index("Set")


score_n_sample_n_set(element_x_sample, set_to_element_)